# CrewAI: Orchestrating Collaborative Agent Systems

**Objective:** This notebook provides a hands-on guide to CrewAI, a framework designed for engineering sophisticated multi-agent systems. You will learn how to create a "crew" of autonomous AI agents that collaborate to achieve complex goals.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** CrewAI's strength lies in its intuitive, high-level abstractions for defining agents with specific roles, assigning them tasks, and defining the workflow (process) they should follow. It simplifies the creation of systems where multiple agents work together, each contributing its specialized skills.

## 1. Setup

First, we'll install the necessary libraries. CrewAI uses LangChain components under the hood. We will also install the `tavily-python` library directly to create a custom search tool, which helps avoid complex dependency issues.

In [28]:
%pip install --upgrade -q crewai langchain_openai python-dotenv tavily-python
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: OPENAI_API_KEY or TAVILY_API_KEY not found. Please check your .env file.")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.28 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## 2. Foundational Crew: The Two-Agent Team

Our first example will be a simple, two-agent crew designed to plan a trip. This will introduce the three core concepts of CrewAI:

-   **`Agent`**: A role-playing entity with a specific `role`, `goal`, and `backstory`.
-   **`Task`**: A specific unit of work to be performed by an agent.
-   **`Crew`**: A collection of agents and tasks, along with a defined `process` for execution.

In [30]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Use a LangChain model as the LLM for the agents
llm = ChatOpenAI(model="gpt-4o")

# Agent 1: The Travel Expert
travel_agent = Agent(
    role='Expert Travel Agent',
    goal='Create a detailed, 3-day itinerary for a trip.',
    backstory='You have 20 years of experience in luxury travel planning and know all the hidden gems.',
    verbose=True,
    llm=llm
)

# Agent 2: The Local Food Critic
food_critic = Agent(
    role='Local Food Critic',
    goal='Recommend the best, most authentic local restaurants for the trip.',
    backstory='You are a famous food blogger who lives in the destination city and is an expert on its culinary scene.',
    verbose=True,
    llm=llm
)

# Task 1: Plan the Itinerary (for the Travel Agent)
plan_itinerary = Task(
    description='Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.',
    expected_output='A markdown file with a day-by-day plan, including morning, afternoon, and evening activities.',
    agent=travel_agent
)

# Task 2: Recommend Restaurants (for the Food Critic)
# This task uses the output of the first task as its context.
recommend_restaurants = Task(
    description='Based on the planned itinerary, recommend one authentic restaurant for each day of the trip.',
    expected_output='A list of 3 restaurant names, each with a brief description and why it fits the itinerary.',
    agent=food_critic,
    context=[plan_itinerary]
)

# Form the crew with a sequential process
trip_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary, recommend_restaurants],
    process=Process.sequential,
    verbose=True
)

print("--- Kicking off the Trip Planning Crew ---")
trip_result = trip_crew.kickoff()

print("\n--- Final Itinerary ---")
print(trip_result)

--- Kicking off the Trip Planning Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 48fec746-9c65-4a12-91f9-c3ff98c44986                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # 3-Day Tokyo Cultural Itinerary                                                                               │
│                                                                                                                 │
│  ## Day 1: Traditional Tokyo                                                                                    │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  - **Senso-ji Temple**: Start your day at Tokyo's oldest temple in Asakusa. Explore the vibrant Nakamise        │
│  Street leading up to the temple for souvenirs and traditional snacks.                                          │
│  - **Asakusa Culture and Tourist Information Center**: Enjoy panoramic views of Asakusa and Tokyo Skytree from  │
│  the observation deck.                                                                                          │
│                                                                                                                 │
│  ### Afternoon                                                                                                  │
│  - **Sumida River Cruise**: Take a leisurely cruise down the Sumida River from Asakusa to Hamarikyu Gardens.    │
│  This offers a unique perspective of Tokyo’s skyline.                                                           │
│  - **Hamarikyu Gardens**: Explore this beautiful Edo-period garden and enjoy a traditional tea ceremony at the  │
│  Nakajima Tea House.                                                                                            │
│                                                                                                                 │
│  ### Evening                                                                                                    │
│  - **Dinner at a Ryotei**: Experience a traditional kaiseki meal at a ryotei (Japanese-style restaurant) in     │
│  the Ginza area.                                                                                                │
│  - **Kabukiza Theatre**: End your day with a kabuki performance at Tokyo's main kabuki theatre. Check the       │
│  schedule in advance as performances vary.                                                                      │
│                                                                                                                 │
│  ## Day 2: Modern Meets Tradition                                                                               │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  - **Meiji Shrine**: Begin with a visit to this Shinto shrine located in a forested area near Harajuku.         │
│  Witness a traditional Shinto wedding if you're lucky.                                                          │
│  - **Harajuku Takeshita Street**: Explore the vibrant youth culture and eclectic shops.                         │
│                                                        

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Day 1: Dinner at Ginza Ryotei "Ginza Kojyu"**                                                             │
│     - **Description**: Ginza Kojyu is a renowned kaiseki restaurant that offers an exquisite multi-course       │
│  dining experience. Located in the luxurious Ginza district, it provides an authentic taste of traditional      │
│  Japanese cuisine with seasonal ingredients beautifully presented.                                              │
│     - **Why it fits the itinerary**: After a day immersed in traditional Tokyo, ending with a kaiseki meal at   │
│  Ginza Kojyu enhances the cultural exploration. It complements the traditional experiences of the day, such as  │
│  the tea ceremony and kabuki performance, by offering diners a true taste of Japanese culinary art.             │
│                                                                                                                 │
│  2. **Day 2: Dinner in Shibuya at "Uobei Shibuya Dogenzaka"**                                                   │
│     - **Description**: Uobei Shibuya Dogenzaka is a modern sushi restaurant that combines the essence of        │
│  traditional sushi with a high-tech ordering system. It's fast, fresh, and fun, making it a perfect fit for     │
│  the vibrant Shibuya atmosphere.                                                                                │
│     - **Why it fits the itinerary**: After exploring the modern and lively areas of Harajuku and Shibuya,       │
│  dining at Uobei adds a modern twist to the day. It provides an interactive dining experience, aligning with    │
│  the theme of modern meets tradition, and prepares you for the breathtaking views from the Shibuya Sky          │
│  observation deck.                                                                                              │
│                                                                                                                 │
│  3. **Day 3: Dinner in Akihabara at "Maidreamin Akihabara"**                                                    │
│     - **Description**: Maidreamin is a themed café in Akihabara, known for its playful and whimsical            │
│  maid-themed dining experience. The café's unique atmosphere offers an insight into Japan's pop culture and is  │
│  a hit among both locals and tourists.                                                                          │
│     - **Why it fits the itinerary**: After a day of cultural immersion and exploring historical sites,          │
│  concluding with a meal at Maidreamin allows visitors to experience the quirky and entertaining side of Tokyo.  │
│  It's an ideal spot to unwind and enjoy the vibrant subculture of Akihabara, especially after visiting the      │
│  electronics and pop culture district.                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7cd0c77b-3335-43c3-bcb8-ed9168a830a8                                                                     │
│  Agent: Local Food Critic                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 48fec746-9c65-4a12-91f9-c3ff98c44986                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Day 1: Dinner at Ginza Ryotei "Ginza Kojyu"**                                               │
│     - **Description**: Ginza Kojyu is a renowned kaiseki restaurant that offers an exquisite multi-course       │
│  dining experience. Located in the luxurious Ginza district, it provides an authentic taste of traditional      │
│  Japanese cuisine with seasonal ingredients beautifully presented.                                              │
│     - **Why it fits the itinerary**: After a day immersed in traditional Tokyo, ending with a kaiseki meal at   │
│  Ginza Kojyu enhances the cultural exploration. It complements the traditional experiences of the day, such as  │
│  the tea ceremony and kabuki performance, by offering diners a true taste of Japanese culinary art.             │
│                                                                                                                 │
│  2. **Day 2: Dinner in Shibuya at "Uobei Shibuya Dogenzaka"**                                                   │
│     - **Description**: Uobei Shibuya Dogenzaka is a modern sushi restaurant that combines the essence of        │
│  traditional sushi with a high-tech ordering system. It's fast, fresh, and fun, making it a perfect fit for     │
│  the vibrant Shibuya atmosphere.                                                                                │
│     - **Why it fits the itinerary**: After exploring the modern and lively areas of Harajuku and Shibuya,       │
│  dining at Uobei adds a modern twist to the day. It provides an interactive dining experience, aligning with    │
│  the theme of modern meets tradition, and prepares you for the breathtaking views from the Shibuya Sky          │
│  observation deck.                                                                                              │
│                                                                                                                 │
│  3. **Day 3: Dinner in Akihabara at "Maidreamin Akihabara"**                                                    │
│     - **Description**: Maidreamin is a themed café in Akihabara, known for its playful and whimsical            │
│  maid-themed dining experience. The café's unique atmosphere offers an insight into Japan's pop culture and is  │
│  a hit among both locals and tourists.                                                                          │
│     - **Why it fits the itinerary**: After a day of cultural immersion and exploring historical sites,          │
│  concluding with a meal at Maidreamin allows visitors to experience the quirky and entertaining side of Tokyo.  │
│  It's an ideal spot to unwind and enjoy the vibrant subculture of Akihabara, especially after visiting the      │
│  electronics and pop culture district.                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Itinerary ---
1. **Day 1: Dinner at Ginza Ryotei "Ginza Kojyu"**
   - **Description**: Ginza Kojyu is a renowned kaiseki restaurant that offers an exquisite multi-course dining experience. Located in the luxurious Ginza district, it provides an authentic taste of traditional Japanese cuisine with seasonal ingredients beautifully presented.
   - **Why it fits the itinerary**: After a day immersed in traditional Tokyo, ending with a kaiseki meal at Ginza Kojyu enhances the cultural exploration. It complements the traditional experiences of the day, such as the tea ceremony and kabuki performance, by offering diners a true taste of Japanese culinary art.

2. **Day 2: Dinner in Shibuya at "Uobei Shibuya Dogenzaka"**
   - **Description**: Uobei Shibuya Dogenzaka is a modern sushi restaurant that combines the essence of traditional sushi with a high-tech ordering system. It's fast, fresh, and fun, making it a perfect fit for the vibrant Shibuya atmosphere.
   - **Why it fits the i

## 3. Adding Tools to a CrewAI Agent

CrewAI agents can use tools to access external information and capabilities. Instead of relying on pre-built wrappers which can cause dependency issues, we can create our own custom tool by wrapping the Tavily Python client. This is a robust and flexible approach.

In [32]:
from crewai.tools import BaseTool
from tavily import TavilyClient

# 1. Create a custom tool by inheriting from BaseTool
class TavilySearchTool(BaseTool):
    name: str = "TavilySearch"
    description: str = "A tool that can be used to search the web with Tavily for up-to-date information."
    
    def _run(self, query: str) -> str:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        response = client.search(query=query, search_depth="basic")
        return response['results']

# Instantiate our custom tool
search_tool = TavilySearchTool()

# 2. Create an agent and assign the custom tool to it
market_researcher = Agent(
    role='Market Research Analyst',
    goal='Find and summarize the latest news about a specific company.',
    backstory='You are a skilled analyst who can quickly find and synthesize financial news.',
    tools=[search_tool], # Assign the custom tool here
    verbose=True,
    llm=llm
)

# 3. Create a task for the agent
research_company_task = Task(
    description='Find the top 3 news headlines for NVIDIA (stock ticker NVDA) from the last 24 hours and summarize them.',
    expected_output='A numbered list of 3 headlines, each followed by a one-sentence summary.',
    agent=market_researcher
)

# 4. Form a single-agent crew to run the task
research_crew = Crew(
    agents=[market_researcher],
    tasks=[research_company_task],
    verbose=True
)

print("--- Kicking off the Research Crew with Tools ---")
research_result = research_crew.kickoff()

print("\n--- Final Research Summary ---")
print(research_result)

--- Kicking off the Research Crew with Tools ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 58f65c4d-154e-477b-976a-189ec3f485ea                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Find the top 3 news headlines for NVIDIA (stock ticker NVDA) from the last 24 hours and summarize them.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to search for the latest news about NVIDIA from the last 24 hours.                    │
│                                                                                                                 │
│  Using Tool: TavilySearch                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Nvidia AI chips worth $1 billion entered China despite US curbs - Nvidia's high-end AI chips, valued at $1  │
│  billion, have made their way into China despite U.S. restrictions, according to a report by the Financial      │
│  Times.                                                                                                         │
│  2. NVIDIA Becomes World's First $4 Trillion Company - NVIDIA has achieved a significant milestone by becoming  │
│  the world's first company to reach a $4 trillion market capitalization, marking its dominance in the AI and    │
│  semiconductor industry.                                                                                        │
│  3. Hexagon Taps NVIDIA Robotics and AI Software to Build and Deploy AEON, a New Humanoid - Hexagon has         │
│  partnered with NVIDIA to utilize its robotics and AI software for the development and deployment of a new      │
│  humanoid robot named AEON, addressing the global labor shortage.                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e53d4838-f9b4-4611-901d-f36da5f9c85a                                                                     │
│  Agent: Market Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 58f65c4d-154e-477b-976a-189ec3f485ea                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. Nvidia AI chips worth $1 billion entered China despite US curbs - Nvidia's high-end AI        │
│  chips, valued at $1 billion, have made their way into China despite U.S. restrictions, according to a report   │
│  by the Financial Times.                                                                                        │
│  2. NVIDIA Becomes World's First $4 Trillion Company - NVIDIA has achieved a significant milestone by becoming  │
│  the world's first company to reach a $4 trillion market capitalization, marking its dominance in the AI and    │
│  semiconductor industry.                                                                                        │
│  3. Hexagon Taps NVIDIA Robotics and AI Software to Build and Deploy AEON, a New Humanoid - Hexagon has         │
│  partnered with NVIDIA to utilize its robotics and AI software for the development and deployment of a new      │
│  humanoid robot named AEON, addressing the global labor shortage.                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Research Summary ---
1. Nvidia AI chips worth $1 billion entered China despite US curbs - Nvidia's high-end AI chips, valued at $1 billion, have made their way into China despite U.S. restrictions, according to a report by the Financial Times.
2. NVIDIA Becomes World's First $4 Trillion Company - NVIDIA has achieved a significant milestone by becoming the world's first company to reach a $4 trillion market capitalization, marking its dominance in the AI and semiconductor industry.
3. Hexagon Taps NVIDIA Robotics and AI Software to Build and Deploy AEON, a New Humanoid - Hexagon has partnered with NVIDIA to utilize its robotics and AI software for the development and deployment of a new humanoid robot named AEON, addressing the global labor shortage.


## 4. Advanced Capability: Hierarchical Process

For more complex workflows, CrewAI offers a `hierarchical` process. In this mode, a manager agent is dynamically nominated to orchestrate the crew, delegating tasks to other agents and synthesizing the final result. This is suitable for problems that require more dynamic coordination.

In [34]:
# Re-using the agents from our first example

# The tasks are the same, but we don't need to specify context
plan_itinerary_h = Task(
    description='Create a 3-day travel itinerary for a trip to Rome, Italy. Focus on historical sites.',
    expected_output='A markdown file with a day-by-day plan.',
    agent=travel_agent
)

recommend_restaurants_h = Task(
    description='Recommend one authentic restaurant for each day of the trip to Rome.',
    expected_output='A list of 3 restaurant names with descriptions.',
    agent=food_critic
)

# Form the crew with a hierarchical process
hierarchical_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary_h, recommend_restaurants_h],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=llm # Specify an LLM for the manager agent
)

print("--- Kicking off the Hierarchical Crew ---")
hierarchical_result = hierarchical_crew.kickoff()

print("\n--- Final Hierarchical Result ---")
print(hierarchical_result)

--- Kicking off the Hierarchical Crew ---


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: What are the must-see historical sites in Rome for a 3-day itinerary?                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **3-Day Itinerary for Historical Sites in Rome**                                                               │
│                                                                                                                 │
│  **Day 1: Ancient Rome Exploration**                                                                            │
│  - **Morning: Colosseum and Roman Forum**                                                                       │
│    - Start your day early at the Colosseum, an iconic symbol of Ancient Rome. Consider purchasing               │
│  skip-the-line tickets in advance or joining a guided tour to learn about the gladiatorial games and the        │
│  history of this monumental amphitheater.                                                                       │
│    - After the Colosseum, explore the Roman Forum nearby. Walk through the ruins and imagine the political,     │
│  social, and economic center of the ancient world. Key sites include the Temple of Saturn, the Arch of Titus,   │
│  and the House of the Vestal Virgins.                                                                           │
│    - **Accessibility:** Both sites are accessible, but be prepared for uneven surfaces.                         │
│    - **Visiting Hours:** Typically open from 8:30 AM to one hour before sunset; check current schedules.        │
│                                                                                                                 │
│  - **Afternoon: Palatine Hill**                                                                                 │
│    - Head to Palatine Hill, one of the most ancient parts of the city and the legendary birthplace of Rome.     │
│  Enjoy panoramic views of the city and visit the ruins of imperial palaces.                                     │
│    - **Special Considerations:** Consider a combined ticket for the Colosseum, Roman Forum, and Palatine Hill   │
│  to save time and money.                                                                                        │
│                                                                                                                 │
│  - **Evening: Capitoline Hill and Capitoline Museums**                                                          │
│    - Visit Capitoline Hill, home to the Capitoline Museums. Discover art and artifacts that trace Rome's        │
│  history from antiquity to the Renaissance.                                                                     │
│    - **Accessibility:** Elevators and ramps are available within the museum.                                    │
│                                                                                                                 │
│  **Day 2: Vatican City and Renaissance Wonders**                                                                │
│  - **Morning: Vatican Museums and Sistine Chapel**                                                              │
│    - Begin with a visit to the Vatican Museums. It's advisable to book tickets in advance to avoid long lines.  │
│  Marvel at the vast art collections and don't miss the breathtaking Sistine Chapel ceiling painted by           │
│  Michelangelo.                                                                                                  │
│    - **Accessibility:** Wheelchair access is available;

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 1 of the trip to Rome, which includes exploring        │
│  Ancient Rome sites like the Colosseum, Roman Forum, Palatine Hill, and Capitoline Hill?                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For an authentic dining experience on Day 1 of your trip to Rome, I recommend visiting "Osteria da             │
│  Fortunata." This charming restaurant is conveniently located near the Ancient Rome sites and offers            │
│  traditional Roman cuisine made from scratch. Known for its fresh pasta and classic dishes like Cacio e Pepe    │
│  and Amatriciana, Osteria da Fortunata captures the essence of Roman dining. The rustic ambiance and the warm   │
│  hospitality of this establishment make it a perfect spot to relax and enjoy a meal after exploring the         │
│  Colosseum, Roman Forum, Palatine Hill, and Capitoline Hill. Be sure to try their homemade desserts to          │
│  complete your historical and culinary experience in Rome.                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 2 of the trip to Rome, which includes visiting         │
│  Vatican Museums, Sistine Chapel, St. Peter's Basilica, and Castel Sant'Angelo?                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For an authentic Roman dining experience close to the Vatican Museums, Sistine Chapel, St. Peter's Basilica,   │
│  and Castel Sant'Angelo, I highly recommend "Ristorante Arlu." Located just a short walk from the Vatican,      │
│  this charming restaurant combines traditional Roman cuisine with a warm, welcoming atmosphere. Their menu      │
│  features classic dishes such as Cacio e Pepe, Saltimbocca alla Romana, and Carciofi alla Giudia (Jewish-style  │
│  artichokes), all served with a focus on fresh, local ingredients. This spot is beloved by locals and offers a  │
│  taste of authentic Roman flavors that will enhance your cultural and culinary exploration of the area. Be      │
│  sure to try their homemade tiramisu for a perfect end to your meal.                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 3 of the trip to Rome, which includes visiting the     │
│  Pantheon, Piazza Navona, Trevi Fountain, Spanish Steps, and Trastevere Neighborhood?                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For an authentic Roman dining experience on Day 3 of your trip, I recommend dining at "Osteria der Belli" in   │
│  the Trastevere neighborhood. This charming osteria is well-known for its traditional Roman cuisine and         │
│  seafood dishes, offering a cozy and welcoming atmosphere that perfectly complements the vibrant and historic   │
│  ambiance of Trastevere. It's conveniently located near your day's itinerary of iconic sites like the           │
│  Pantheon, Piazza Navona, Trevi Fountain, and the Spanish Steps, making it an ideal spot to unwind after a day  │
│  of exploration. Be sure to try their classic Roman pasta dishes like Cacio e Pepe or Carbonara for a true      │
│  taste of Rome. Make a reservation, as this popular spot tends to fill up quickly, especially in the evening.   │
│  Enjoy your meal and the lovely Trastevere vibe!                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Hierarchical Result ---
1. **Day 1: Osteria da Fortunata**
   - Located near the Ancient Rome sites, Osteria da Fortunata offers traditional Roman cuisine made from scratch. Known for its fresh pasta and classic dishes like Cacio e Pepe and Amatriciana, the restaurant captures the essence of Roman dining. The rustic ambiance and warm hospitality make it a perfect spot to relax and enjoy a meal after exploring the Colosseum, Roman Forum, Palatine Hill, and Capitoline Hill. Be sure to try their homemade desserts to complete your historical and culinary experience in Rome.

2. **Day 2: Ristorante Arlu**
   - Situated just a short walk from the Vatican, Ristorante Arlu combines traditional Roman cuisine with a warm, welcoming atmosphere. Their menu features classic dishes such as Cacio e Pepe, Saltimbocca alla Romana, and Carciofi alla Giudia (Jewish-style artichokes), all served with a focus on fresh, local ingredients. This restaurant is beloved by locals and offers a taste of

## Lab Conclusion

In this lab, you've learned the fundamentals of CrewAI. You've seen how to define agents with distinct roles, create tasks for them to perform, and assemble them into a crew that can work together sequentially or be managed hierarchically.

**Key Takeaways:**
- CrewAI is excellent for problems that can be broken down into distinct roles and responsibilities.
- The `Agent`, `Task`, and `Crew` abstractions provide a clear and intuitive way to structure multi-agent systems.
- Creating custom tools by inheriting from `BaseTool` is a reliable way to add capabilities and avoid dependency conflicts.
- The `hierarchical` process allows for more dynamic, manager-led coordination for complex, non-linear tasks.